<font size=6 color=red>ACCによる定速走行データをセグメントごとに集約し、VAVRと比較する関数</font>

In [1]:
import os
import re
from pathlib import Path
from os import path
import json
import time
import pyodbc
import webbrowser
import shutil
import csv
import pandas as pd
#import folium
import datetime
import pyodbc
import numpy as np
import math
import datetime
import warnings
import math
import matplotlib.pyplot as plt
%matplotlib inline

warnings.simplefilter('ignore')
pd.set_option('display.max_rows',250)
from sklearn.metrics import mean_absolute_error,r2_score
# ヒュベニの公式
# https://butter-tiger.hatenablog.com/entry/2020/08/20/222650
POLE_RADIUS = 6356752  # 極半径(短半径)
EQUATOR_RADIUS = 6378137  # 赤道半径(長半径)
E = 0.081819191042815790  # 離心率
E2 = 0.006694380022900788  # 離心率の２乗

In [2]:
def pearson_corr_2(x, y):
    x_diff = x - np.mean(x)
    y_diff = y - np.mean(y)
    r2_c = (np.dot(x_diff, y_diff) / (np.sqrt(sum(x_diff ** 2)) * np.sqrt(sum(y_diff ** 2)))) ** 2
    return r2_c


In [3]:
def distance(_lat1, _long1, _lat2, _long2):
    lat1 = math.radians(_lat1)
    long1 = math.radians(_long1)
    lat2 = math.radians(_lat2)
    long2 = math.radians(_long2)
    m_lat = (lat1 + lat2) / 2  # 平均緯度
    d_lat = abs(lat1 - lat2)  # 緯度差
    d_lon = abs(long1 - long2)  # 経度差
    W = math.sqrt(1 - E2 * math.pow(math.sin(m_lat), 2))
    M = EQUATOR_RADIUS * (1 - E2) / math.pow(W, 3)  # 子午線曲率半径
    N = EQUATOR_RADIUS / W  # 卯酉線曲率半径
    # d = math.sqrt(math.pow(M*d_lat,2) + math.pow(N*d_lon*math.cos(m_lat),2) + math.pow(point_a.altitude-point_b.altitude,2))
    d = math.sqrt(math.pow(M * d_lat, 2) + math.pow(N * d_lon * math.cos(m_lat), 2))
    return d

In [4]:
def get_segment_datas(semanticlink_id):
    driver='{SQL Server}'
    server = 'ecologdatabasesystem2030'
    database = 'ECOLOGDdatabaseppp2030'
    trusted_connection='yes'
    connect= pyodbc.connect('DRIVER='+driver+';SERVER='+server+';DATABASE='+database+';PORT=1433;Trusted_Connection='+trusted_connection+';')
    cursor = connect.cursor()
    
    sql = """select SEMANTIC_LINK_ID
                   ,A.SEGMENT_ID
                   ,START_LATITUDE
                   ,START_LONGITUDE
                   ,END_LATITUDE
                  ,END_LONGITUDE
            from SEMANTIC_LINKS_SEGMENTS_50M as A inner join SEGMENTS_---_50M as B on A.SEGMENT_ID = B.SEGMENT_ID
            where SEMANTIC_LINK_ID = {}
            """.format(str(semanticlink_id))

    cursor.execute(sql) 
    rows = cursor.fetchall()
    df_rows = pd.DataFrame({'SEMANTIC_LINK_ID':[],
                            'SEGMENT_ID':[],
                            'START_LATITUDE':[],
                            'START_LONGITUDE':[],
                            'END_LATITUDE':[],
                            'END_LONGITUDE':[]})
    for i in range(len(rows)):
        df_rows.loc[i,:] = list(rows[i])
    
    cursor.close()
    connect.close()
    return df_rows



In [ ]:
print('input semanticlink_id:')
semanticlink_id = int(input())

print('input acc setting speed:')
acc_setting_speed = int(input())

print('input AC_setting(0 or 250):')
AC_setting = int(input())

if semanticlink_id == ***:
    ACC_csv_path =  rf"{os.getcwd()}\input\ACC\***_acc_setting_speed{acc_setting_speed}_AC_setting{AC_setting}W.csv"
    VCVR_csv_path = rf"{os.getcwd()}\input\VCVR_ecolog\virtual_run_***_acc_setting_speed{acc_setting_speed}.xlsx"

else:
    ACC_csv_path =  rf"{os.getcwd()}\input\ACC\{semanticlink_id}_acc_setting_speed{acc_setting_speed}_AC_setting{AC_setting}W.csv"
    VCVR_csv_path = rf"{os.getcwd()}\input\VCVR_ecolog\virtual_run_{semanticlink_id}_acc_setting_speed{acc_setting_speed}.xlsx"

df_ACC = pd.read_csv(ACC_csv_path)[['Trip_id','Lat','Lon','PACK_Wh']]
df_VCVR = pd.read_excel(VCVR_csv_path)

df_segments_data = get_segment_datas(semanticlink_id)
START_LATITUDE = df_segments_data.START_LATITUDE.iloc[0]
START_LONGITUDE = df_segments_data.START_LONGITUDE.iloc[0]
df_distance_list_of_segments = df_segments_data[['SEMANTIC_LINK_ID','SEGMENT_ID']]

distance_from_start = **
distance_list = []
for i in range(len(df_distance_list_of_segments)):
    distance_list.append(distance_from_start)
    distance_from_start = distance_from_start + **
df_distance_list_of_segments['distance_from_start'] = distance_list

###################################
#part of ACC
trip_list = df_ACC.Trip_id[~df_ACC.Trip_id.duplicated(keep = 'first')].to_list()
ACC_distance_list = []
for trip in trip_list:
    ACC_distance_from_start = 0
    for i in range(len(df_ACC[df_ACC.Trip_id.isin([trip])])):
        if i<1:
            ACC_distance_from_start = distance(START_LATITUDE,
                                               START_LONGITUDE,
                                               df_ACC[df_ACC.Trip_id.isin([trip])].Lat.iloc[0],
                                               df_ACC[df_ACC.Trip_id.isin([trip])].Lon.iloc[0])+ACC_distance_from_start
            ACC_distance_list.append(ACC_distance_from_start)
        else:
            ACC_distance_from_start = distance(df_ACC[df_ACC.Trip_id.isin([trip])].Lat.iloc[i-1],
                                               df_ACC[df_ACC.Trip_id.isin([trip])].Lon.iloc[i-1],
                                               df_ACC[df_ACC.Trip_id.isin([trip])].Lat.iloc[i],
                                               df_ACC[df_ACC.Trip_id.isin([trip])].Lon.iloc[i]) +ACC_distance_from_start
            ACC_distance_list.append(ACC_distance_from_start)
            
df_ACC['distance_from_start'] = ACC_distance_list

df_ACC['segment'] = 0
acc_cla_list = []
for trip in trip_list:
    acc_cla = 0
    for i in range(len(df_ACC[df_ACC.Trip_id.isin([trip])])):
        if df_ACC[df_ACC.Trip_id.isin([trip])].distance_from_start.iloc[i]<=df_distance_list_of_segments.distance_from_start.iloc[-1]:
            for j in range(len(df_distance_list_of_segments)):
                if df_distance_list_of_segments.distance_from_start.iloc[j]>df_ACC[df_ACC.Trip_id.isin([trip])].distance_from_start.iloc[i]:
                    if df_distance_list_of_segments.distance_from_start.iloc[j-1]<df_ACC[df_ACC.Trip_id.isin([trip])].distance_from_start.iloc[i]:
                        acc_cla = df_distance_list_of_segments.SEGMENT_ID.iloc[j]
                        df_ACC.segment.iloc[df_ACC[df_ACC.Trip_id.isin([trip])].iloc[i].name] = acc_cla

        else:
            acc_cla = df_distance_list_of_segments.SEGMENT_ID.iloc[-1]
            df_ACC.segment.iloc[df_ACC[df_ACC.Trip_id.isin([trip])].iloc[i].name] = acc_cla

###################################
#part of VCVR     

df_VCVR = df_VCVR[['LATITUDE','LONGITUDE','ConsumedEnergy']]

VCVR_distance_list = []
VCVR_distance_from_start = 0
for i in range(len(df_VCVR)):
    if i<1:
        VCVR_distance_from_start = distance(START_LATITUDE,
                                           START_LONGITUDE,
                                           df_VCVR.LATITUDE.iloc[0],
                                           df_VCVR.LONGITUDE.iloc[0])+VCVR_distance_from_start
        VCVR_distance_list.append(VCVR_distance_from_start)
    else:
        VCVR_distance_from_start = distance(df_VCVR.LATITUDE.iloc[i-1],
                                           df_VCVR.LONGITUDE.iloc[i-1],
                                           df_VCVR.LATITUDE.iloc[i],
                                           df_VCVR.LONGITUDE.iloc[i]) +VCVR_distance_from_start
        VCVR_distance_list.append(VCVR_distance_from_start)
            
df_VCVR['distance_from_start'] = VCVR_distance_list

df_VCVR['segment'] = 0
VCVR_cla_list = []
VCVR_cla = 0
for i in range(len(df_VCVR)):
    if df_VCVR.distance_from_start.iloc[i]<=df_distance_list_of_segments.distance_from_start.iloc[-1]:
        for j in range(len(df_distance_list_of_segments)):
            if df_distance_list_of_segments.distance_from_start.iloc[j]>df_VCVR.distance_from_start.iloc[i]:
                if df_distance_list_of_segments.distance_from_start.iloc[j-1]<df_VCVR.distance_from_start.iloc[i]:
                    VCVR_cla = df_distance_list_of_segments.SEGMENT_ID.iloc[j]
                    df_VCVR.segment.iloc[i] = VCVR_cla
                elif j == 0:
                    VCVR_cla = df_distance_list_of_segments.SEGMENT_ID.iloc[j]
                    df_VCVR.segment.iloc[i] = VCVR_cla
                    

    else:
        VCVR_cla = df_distance_list_of_segments.SEGMENT_ID.iloc[-1]
        df_VCVR.segment.iloc[i] = VCVR_cla
        

# road3 ACC_35km/h

In [ ]:
df_road3_segment = df_distance_list_of_segments
df_road3_VCVR = df_VCVR
df_road3_VCVR_for_calculate = df_road3_VCVR.groupby('segment').ConsumedEnergy.agg('mean').reset_index(drop = True)*1000*1000/50
df_road3_ACC = df_ACC
df_road3_ACC_for_result = df_road3_ACC[df_road3_ACC.segment.isin(df_road3_VCVR.segment)]
df_road3_ACC_for_calculate = df_road3_ACC_for_result.groupby('segment').PACK_Wh.agg( 'median').reset_index(drop = True)*1000/50
x_sticker = np.array(range(len(df_road3_VCVR_for_calculate)))*50
r2_road3 = pearson_corr_2(df_road3_VCVR_for_calculate,df_road3_ACC_for_calculate)
plt.plot(x_sticker,df_road3_VCVR_for_calculate,color = 'blue',alpha =0.6,label = 'VCVR')
plt.plot(x_sticker,df_road3_ACC_for_calculate,linestyle = ':',color = 'red',alpha =0.6,label = 'ACC')
plt.text(x_sticker.max()*0.6,df_road3_VCVR_for_calculate.max()*0.9,
         'R^2: {}'.format(math.floor(r2_road3 * 10 ** 3) / (10 ** 3))
         ,fontsize = 10,color = 'r') 
plt.legend(loc='best')
plt.xlabel('Distance from start (m)')
plt.ylabel('Energy consumption (Wh/km)')

# semantic_link:road1 Acc_setting_speed:85km/h

In [ ]:
df_road1_distance_list_of_segments = df_distance_list_of_segments
df_road1_VCVR = df_VCVR
df_road1_VCVR_for_calculate = df_road1_VCVR.groupby('segment').ConsumedEnergy.agg('mean').reset_index(drop = True).iloc[3:105]*1000*1000/50
df_road1_ACC = df_ACC
after_df_road1_ACC_for_calculate = df_road1_ACC.groupby('segment').PACK_Wh.agg( 'median').reset_index(drop = True).iloc[4:106]*1000/50
x_sticker = np.array(range(len(df_road1_VCVR_for_calculate)))*50
after_r2_road1 = pearson_corr_2(df_road1_VCVR_for_calculate,after_df_road1_ACC_for_calculate)
plt.plot(x_sticker,df_road1_VCVR_for_calculate,color = 'blue',alpha =0.6,label = 'VCVR')
plt.plot(x_sticker,after_df_road1_ACC_for_calculate,linestyle = ':',color = 'red',alpha =0.6,label = 'ACC')
plt.text(x_sticker.max()*0.65,df_road1_VCVR_for_calculate.max()*0.9,
         'R^2: {}'.format(math.floor(after_r2_road1 * 10 ** 3) / (10 ** 3))
         ,fontsize = 10,color = 'r') 
plt.legend(loc='best')
plt.xlabel('Distance from start (m)')
plt.ylabel('Energy consumption (Wh/km)')

# semantic_link : road2 Acc_setting_speed:85km/h

In [ ]:
df_road2_distance_list_of_segments = df_distance_list_of_segments
df_road2_VCVR = df_VCVR
df_road2_ACC = df_ACC
df_road2_VCVR_for_result = df_road2_VCVR[df_road2_VCVR.segment.isin(df_road2_ACC.segment)]
df_road2_VCVR_for_calculate = df_road2_VCVR_for_result.groupby('segment').ConsumedEnergy.agg('mean').reset_index(drop = True).iloc[:75]*1000*1000/50
after_df_road2_ACC_for_calculate = df_road2_ACC.groupby('segment').PACK_Wh.agg( 'median').reset_index(drop = True).iloc[1:76]*1000/50
x_sticker = np.array(range(len(df_road2_VCVR_for_calculate)))*50
after_r2_road2 = pearson_corr_2(after_df_road2_ACC_for_calculate,df_road2_VCVR_for_calculate)
plt.plot(x_sticker,df_road2_VCVR_for_calculate,color = 'blue',alpha =0.6,label = 'VCVR')
plt.plot(x_sticker,after_df_road2_ACC_for_calculate,linestyle = ':',color = 'red',alpha =0.6,label = 'ACC')
plt.text(x_sticker.max()*0.64,df_road2_VCVR_for_calculate.max()*0.95,
         'R^2: {}'.format(math.floor(after_r2_road2 * 10 ** 3) / (10 ** 3))
         ,fontsize = 10,color = 'r') 
plt.legend(loc='best')
plt.xlabel('Distance from start (m)')
plt.ylabel('Energy consumption (Wh/km)')